<a href="https://colab.research.google.com/github/moch1996/Vaycold/blob/main/Recommend_Algorithm/%234.%EC%9D%B4%EC%9B%83%EA%B8%B0%EB%B0%98_%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이웃기반 협업필터링
 - 협업필터링 종류 : 모델기반, 이웃기반, 하이브리드 딥러닝 기반 
 - 이웃기반 협업필터링 -> 유사도 계산, 아이템기반, 유저기반 ... 

## 라이브러리 불러오기

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


## 데이터 불러오기

In [3]:
path = '/content/drive/MyDrive/fastcampus/recommend/movielens'
ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')
movies_df = pd.read_csv(os.path.join(path, 'movies.csv'), index_col='movieId', encoding='utf-8')
tags_df = pd.read_csv(os.path.join(path, 'tags.csv'), encoding='utf-8')

In [4]:
print(ratings_df.shape)
print(ratings_df.head())

(100836, 4)
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [5]:
train_df, test_df = train_test_split(ratings_df, test_size = 0.2, random_state = 1234)

## Sparse matrix 만들기

 - sparse matrix = (user, movie)

In [6]:
# Sparse Matrix 만들기
# Sparse Matrix란 듬성듬성 있는 매트릭스임.
# pandas dataframe의 unstack 사용

In [7]:
print(train_df.shape)
print(train_df.head())

(80668, 4)
       userId  movieId  rating   timestamp
95713     600     5943     3.0  1237714356
61560     407     2571     5.0  1424349171
77204     482     8958     4.0  1105397126
93367     599     2322     2.5  1498515283
90892     590     2959     3.5  1258416553


In [8]:
sparse_matrix = train_df.groupby('movieId').apply(lambda x : pd.Series(x['rating'].values, index = x['userId'])).unstack()
sparse_matrix

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,3.5,4.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.0,NaN,4.0,NaN,4.0,3.0,4.0,2.5,NaN,2.5,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,3.5,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,2.5,4.0,NaN,4.0,NaN,NaN,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# unstack을 적용안했을 시
sparse_matrix2 = train_df.groupby('movieId').apply(lambda x : pd.Series(x['rating'].values, index = x['userId']))
sparse_matrix2 # 해석을 해보자.
               # groupby('movieId) 를 하였으므로 movieId로 묶는 다는 소리임 그런데 어떻게 묶느냐?? 뒤에 apply함수를 이용해서 묶을거임.
               # apply 함수를 보자 
               

movieId  userId
1        46        5.0
         161       4.0
         453       5.0
         604       3.0
         169       4.5
                  ... 
193573   184       4.0
193579   184       3.5
193581   184       4.0
193587   184       3.5
193609   331       4.0
Length: 80668, dtype: float64

In [ ]:
train_df[train_df.userId==46].head(2)

,userId,movieId,rating,timestamp
6876,46,1,5.0,834787906
6901,46,318,4.0,834787757


In [ ]:
train_df[train_df.movieId==1].rating

6876     5.0
23663    4.0
70844    5.0
97043    3.0
24321    4.5
        ... 
81082    4.0
68337    4.0
71257    5.0
76227    3.0
19179    4.0
Name: rating, Length: 169, dtype: float64

In [ ]:
train_df[train_df.movieId==1].rating.values

array([5. , 4. , 5. , 3. , 4.5, 3.5, 4. , 4.5, 4.5, 4. , 4. , 4.5, 3. ,
       4. , 3.5, 4. , 4. , 3.5, 3. , 4. , 4. , 3.5, 4. , 5. , 5. , 4.5,
       4. , 2. , 4. , 4. , 4. , 4.5, 2.5, 4.5, 4. , 5. , 4. , 3. , 3. ,
       2. , 4. , 4. , 5. , 4. , 2. , 5. , 4. , 3. , 0.5, 4. , 5. , 5. ,
       3.5, 4. , 3.5, 4. , 3. , 3. , 4. , 4.5, 5. , 4.5, 3. , 4. , 4. ,
       4. , 4. , 5. , 3.5, 3. , 3. , 5. , 5. , 4.5, 4. , 2.5, 2. , 2.5,
       5. , 4. , 2. , 2. , 4. , 5. , 4. , 3.5, 4. , 4. , 4.5, 4.5, 4. ,
       4. , 4. , 4. , 5. , 3. , 3. , 3. , 4. , 3.5, 4. , 5. , 4. , 4. ,
       4. , 5. , 5. , 4. , 5. , 3. , 4. , 5. , 4. , 3.5, 4. , 4. , 4. ,
       4. , 5. , 2.5, 4. , 3. , 3.5, 4. , 5. , 3. , 4. , 4. , 2.5, 4.5,
       5. , 5. , 5. , 5. , 3.5, 3.5, 4.5, 3. , 5. , 3.5, 4. , 3.5, 5. ,
       4. , 3. , 4. , 3. , 4. , 4. , 3. , 1.5, 3. , 2.5, 3. , 4. , 5. ,
       4.5, 5. , 4. , 4. , 5. , 5. , 4. , 3. , 4. , 4. , 5. , 3. , 4. ])

In [ ]:
train_df[train_df.movieId==1].userId

6876      46
23663    161
70844    453
97043    604
24321    169
        ... 
81082    514
68337    443
71257    456
76227    480
19179    124
Name: userId, Length: 169, dtype: int64

In [9]:
sparse_matrix

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,3.5,4.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.0,NaN,4.0,NaN,4.0,3.0,4.0,2.5,NaN,2.5,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,3.5,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,2.5,4.0,NaN,4.0,NaN,NaN,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# 결측값을 0으로 주는 것이 맞을까??
# 결측값이라는 것은 안보았다는 것을 의미하는데, 이렇게 하면 0점을 준것이랑 같게 된것이지 않을까

## 코사인 유사도 활용하기

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

def cossim_matrix(a,b) :
  cossim_values = cosine_similarity(a.values, b.values)
  cossim_df = pd.DataFrame(data=cossim_values,columns = a.index.values, index = a.index)

  return cossim_df

### Item-Based

In [12]:
item_sparse_matrix = sparse_matrix.fillna(0)

In [15]:
item_cossim_df = cossim_matrix(item_sparse_matrix,item_sparse_matrix)
item_cossim_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,184257,184349,184471,184721,184791,184931,184987,184997,185033,185135,185435,185585,187031,187541,187593,187595,187717,188189,188301,188675,188797,189043,189111,189333,189547,190183,190207,190209,190213,190215,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.304336,0.267816,0.040259,0.221228,0.266544,0.149392,0.132943,0.182044,0.296838,0.257709,0.139632,0.113701,0.164856,0.103301,0.242000,0.246986,0.101015,0.251669,0.131482,0.261433,0.182307,0.164879,0.222668,0.266991,0.140973,0.108473,0.109745,0.181797,0.053777,0.162647,0.422169,0.402208,0.266803,0.116007,0.297235,0.0,0.085053,0.021893,0.114641,...,0.000000,0.057923,0.045449,0.028962,0.090051,0.077231,0.077231,0.077231,0.077231,0.045207,0.0,0.05933,0.040932,0.0,0.060720,0.082251,0.028962,0.028962,0.061785,0.000000,0.077231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.304336,1.000000,0.226138,0.052482,0.154783,0.209716,0.189420,0.068012,0.027945,0.303157,0.244600,0.156918,0.073411,0.083439,0.158038,0.205833,0.146949,0.193136,0.458342,0.092049,0.235169,0.142401,0.052102,0.182855,0.057490,0.139123,0.037172,0.115826,0.189215,0.000000,0.152457,0.258012,0.324845,0.147365,0.119390,0.336617,0.0,0.078364,0.029729,0.092547,...,0.000000,0.000000,0.165209,0.000000,0.050024,0.000000,0.000000,0.000000,0.000000,0.156683,0.0,0.00000,0.000000,0.0,0.069710,0.136606,0.000000,0.000000,0.000000,0.110117,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.267816,0.226138,1.000000,0.000000,0.306435,0.245555,0.368724,0.168267,0.253679,0.178219,0.235708,0.122641,0.149451,0.138664,0.052048,0.236722,0.188948,0.203784,0.244938,0.073150,0.229087,0.223028,0.115734,0.247492,0.198245,0.127082,0.238175,0.114782,0.110742,0.000000,0.198132,0.233039,0.151203,0.239877,0.119628,0.191147,0.0,0.123729,0.103266,0.208362,...,0.000000,0.000000,0.000000,0.000000,0.089110,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.096418,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.040259,0.052482,0.000000,1.000000,0.095673,0.068708,0.205962,0.000000,0.000000,0.044835,0.144579,0.000000,0.000000,0.161852,0.000000,0.000000,0.125440,0.000000,0.025580,0.000000,0.155352,0.069693,0.000000,0.000000,0.185360,0.051463,0.000000,0.000000,0.000000,0.000000,0.187215,0.050263,0.064871,0.112260,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.221228,0.154783,0.306435,0.095673,1.000000,0.238683,0.343507,0.204088,0.222925,0.150729,0.215188,0.198332,0.110459,0.153343,0.000000,0.156847,0.226190,0.128029,0.194406,0.046290,0.125510,0.176078,0.117994,0.246658,0.267305,0.089126,0.260605,0.049500,0.163221,0.000000,0.275988,0.222996,0.181409,0.198536,0.169278,0.182216,0.0,0.083955,0.000000,0.240221,...,0.178434,0.000000,0.000000,0.000000,0.054040,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.058472,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193573,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [16]:
train_df.head()

,userId,movieId,rating,timestamp
95713,600,5943,3.0,1237714356
61560,407,2571,5.0,1424349171
77204,482,8958,4.0,1105397126
93367,599,2322,2.5,1498515283
90892,590,2959,3.5,1258416553


In [20]:
# train_df 에 포함된 userID를 계산에 반영 
userId_grouped = train_df.groupby('userId')

In [22]:
# 예측 결과를 넣을 데이터 프레임을 만들자.
# index : userId, columns :total movieId
item_prediction_result_df = pd.DataFrame(index = list(userId_grouped.indices.keys()),columns = item_sparse_matrix.index)
item_prediction_result_df

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,184257,184349,184471,184721,184791,184931,184987,184997,185033,185135,185435,185585,187031,187541,187593,187595,187717,188189,188301,188675,188797,189043,189111,189333,189547,190183,190207,190209,190213,190215,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
for userId, group in tqdm(userId_grouped):
    # user가 rating한 movieId * 전체 movieId
    user_sim = item_cossim_df.loc[group['movieId']]
    # user가 rating한 movieId * 1
    user_rating = group['rating']
    # 전체 movieId * 1
    sim_sum = user_sim.sum(axis=0)

    # userId의 전체 rating predictions (8938 * 1)
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    item_prediction_result_df.loc[userId] = pred_ratings

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [24]:
item_prediction_result_df.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,184257,184349,184471,184721,184791,184931,184987,184997,185033,185135,185435,185585,187031,187541,187593,187595,187717,188189,188301,188675,188797,189043,189111,189333,189547,190183,190207,190209,190213,190215,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
1,4.20279,4.18783,4.17609,3.03272,4.04561,4.24019,3.9933,3.90547,3.70279,4.18855,4.16638,3.96413,4.07492,3.89619,3.81881,4.2132,4.16727,4.13229,4.16556,4.1066,4.16574,4.086,4.14826,4.13404,4.13482,4.08018,3.71706,4.0126,4.19837,3.82467,4.09918,4.21487,4.21771,4.13656,4.03317,4.17642,0.863706,4.00858,4.03026,4.09802,...,2.81089,1.78243,3.99862,3.18195,4.07664,3.62958,3.62958,3.21241,3.62958,3.59316,3.74393,3.88679,3.12593,2.2081,3.96647,4.15315,3.18195,3.18195,3.24997,3.83627,3.62958,0.979003,0.979003,0,1.45119,2.40757,0.979003,0.979003,0.979003,0.979003,0.979003,0,0,0,0,0,0,0,0,1.38273
2,3.19261,3.15207,2.46138,0.280903,2.61916,2.9754,1.85877,1.68768,1.09268,3.02235,2.54129,2.51385,1.61082,0.758551,1.63399,3.18457,2.53038,2.90807,3.22072,2.92756,2.40354,2.14887,2.35921,2.71687,2.21494,2.23787,1.19091,1.83372,2.29285,0.196298,2.65615,3.05432,2.98765,2.38185,2.62798,2.91054,0,1.71507,1.77847,2.03524,...,2.61665,0,2.69157,1.20283,2.21043,1.95442,1.95442,1.91231,1.95442,2.76609,2.94477,2.44461,1.33566,1.90164,3.10808,2.94114,1.20283,1.20283,2.26655,2.28051,1.95442,0.387743,0.387743,1.70885,0.362692,1.91228,0.387743,0.387743,0.387743,0.387743,0.387743,1.70885,1.70885,1.70885,1.70885,1.70885,1.70885,1.70885,1.70885,2.32666
3,1.32315,1.20446,1.39635,0.217016,0.819716,1.59889,0.802514,0.755008,0.86708,1.55365,0.942893,1.34004,0.620453,0.450938,0.886356,1.34587,0.85248,1.39133,1.26625,0.999873,1.08345,1.13544,0.846797,1.04494,0.862301,0.817965,0.377594,0.406655,1.5742,1.38647,1.15163,1.46737,1.11338,0.947678,1.03943,1.18457,0,0.873352,0.950293,0.834913,...,0.0383185,0.224785,1.08515,0,0.807464,0.715827,0.715827,0.458293,0.715827,0.55052,0.314237,0.822655,0.469606,0.034753,0.784118,1.03524,0,0,0.397776,0.619607,0.715827,0,0,0,0.213525,0.115378,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3.48994,3.48805,3.38864,3.15017,3.30488,3.50267,3.31794,2.90629,3.03413,3.41697,3.42383,3.2559,3.18772,3.38452,3.20826,3.46465,3.51254,3.44664,3.41673,3.30509,3.5149,3.26365,3.33294,3.42041,3.46056,3.48881,3.06763,3.43736,3.58361,3.47541,3.26032,3.47623,3.49028,3.51451,3.43002,3.46574,0.492723,3.33497,3.31457,3.4271,...,2.43625,1.63485,3.11562,3.34077,3.24972,2.96916,2.96916,2.62972,2.96916,2.7532,3.08765,3.18312,2.56696,1.22959,3.22873,3.32863,3.34077,3.34077,2.62846,3.23077,2.96916,1.0335,1.0335,0.505535,0.810046,1.89312,1.0335,1.0335,1.0335,1.0335,1.0335,0.505535,0.505535,0.505535,0.505535,0.505535,0.505535,0.505535,0.505535,1.963
5,3.27916,3.1278,2.99947,2.56582,2.94179,3.16869,3.00793,2.67809,2.27668,3.01557,3.12409,2.23444,2.59869,3.02734,2.68661,3.08962,3.16745,2.82838,3.00623,2.50037,3.22707,2.88812,2.82624,2.87902,3.15316,2.91281,2.45999,2.90752,3.05304,2.43656,2.9588,3.18185,3.22697,3.31433,2.28735,3.17345,0.158787,2.73675,2.10767,2.91458,...,0.854196,0.33135,2.04168,1.80091,2.33965,1.83931,1.83931,1.53293,1.83931,1.65368,1.70121,2.0751,1.23932,0.318296,2.13799,2.66672,1.80091,1.80091,1.52717,1.83851,1.83931,0.391638,0.391638,0,0.460137,0.680643,0.391638,0.391638,0.391638,0.391638,0.391638,0,0,0,0,0,0,0,0,0.562705
6,3.58597,3.57233,3.53615,3.33866,3.51444,3.5477,3.53826,3.42057,3.3406,3.55275,3.57423,3.32095,3.4276,3.48652,3.50615,3.52936,3.53836,3.42938,3.523,3.3589,3.54642,3.50513,3.49329,3.4643,3.5083,3.50719,3.46573,3.52401,3.40919,2.98586,3.50899,3.54346,3.6113,3.5479,3.39448,3.59635,0.343524,3.44129,3.23668,3.44547,...,2.32125,1.06784,3.10313,1.82397,3.22679,2.24826,2.24826,1.7323,2.24826,2.48076,2.90869,2.71012,1.81119,0.883528,3.08411,3.29296,1.82397,1.82397,1.82883,2.57412,2.24826,0.446497,0.446497,0,0.289115,1.14777,0.446497,0.446497,0.446497,0.446497,0

### user-based

In [25]:
user_sparse_matrix = sparse_matrix.fillna(0).transpose()

In [26]:
user_sparse_matrix

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,184257,184349,184471,184721,184791,184931,184987,184997,185033,185135,185435,185585,187031,187541,187593,187595,187717,188189,188301,188675,188797,189043,189111,189333,189547,190183,190207,190209,190213,190215,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,4.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,2.0,0.0,3.5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.5,3.5,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
user_cossim_df = cossim_matrix(user_sparse_matrix,user_sparse_matrix)
user_cossim_df.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.016665,0.070570,0.160438,0.075410,0.087404,0.123664,0.069225,0.037416,0.009900,0.116869,0.010900,0.097073,0.097351,0.160905,0.130733,0.233488,0.159436,0.229175,0.106378,0.134026,0.042174,0.080940,0.118462,0.057098,0.074305,0.176682,0.154508,0.115075,0.095219,0.116415,0.096891,0.116097,0.060039,0.018472,0.033715,0.080014,0.103986,0.265307,0.062876,...,0.068639,0.092376,0.193911,0.059231,0.038337,0.027807,0.259536,0.0,0.127610,0.196391,0.032494,0.060829,0.041263,0.061662,0.118403,0.065398,0.079916,0.152958,0.049672,0.254597,0.084700,0.103389,0.142405,0.099736,0.068952,0.104889,0.232882,0.014377,0.226264,0.208385,0.056163,0.124896,0.162324,0.050852,0.101583,0.128198,0.240652,0.225897,0.063984,0.095228
2,0.016665,1.000000,0.000000,0.004295,0.020560,0.030996,0.027726,0.000000,0.000000,0.057112,0.050529,0.000000,0.000000,0.019706,0.094221,0.042154,0.080419,0.157239,0.014791,0.016695,0.076210,0.079124,0.000000,0.111357,0.181943,0.000000,0.000000,0.033860,0.096737,0.066817,0.000000,0.021255,0.038112,0.021178,0.000000,0.070899,0.034344,0.030872,0.000000,0.020290,...,0.000000,0.039103,0.124467,0.040580,0.000000,0.000000,0.000000,0.0,0.000000,0.050032,0.130993,0.149300,0.000000,0.000000,0.094519,0.080066,0.000000,0.000000,0.032529,0.050973,0.042262,0.036289,0.053546,0.015933,0.000000,0.095991,0.014417,0.000000,0.079978,0.028680,0.151666,0.019379,0.006645,0.000000,0.000000,0.028180,0.000000,0.046286,0.033522,0.090288
3,0.070570,0.000000,1.000000,0.002677,0.000000,0.003477,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017739,0.037489,0.003184,0.013792,0.015762,0.007023,0.003222,0.000488,0.003862,0.003162,0.000000,0.000000,0.002843,0.012381,0.007261,0.000000,0.004728,0.015895,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002246,0.003035,...,0.009987,0.007018,0.000000,0.000000,0.000000,0.037043,0.003330,0.0,0.000000,0.022490,0.005126,0.000000,0.000000,0.000000,0.000000,0.000000,0.005185,0.000000,0.000000,0.025883,0.000000,0.000000,0.003586,0.002482,0.000000,0.011629,0.019767,0.000000,0.030780,0.003817,0.002878,0.002174,0.027609,0.000000,0.000000,0.012090,0.000000,0.023927,0.000000,0.018332
4,0.160438,0.004295,0.002677,1.000000,0.121648,0.093634,0.106495,0.047930,0.000000,0.034281,0.051343,0.062434,0.068593,0.047313,0.056951,0.153868,0.115363,0.108863,0.173788,0.092159,0.035909,0.053011,0.069875,0.055751,0.021803,0.073772,0.100712,0.125315,0.062167,0.059498,0.075626,0.177156,0.134276,0.036773,0.050788,0.091729,0.072888,0.054797,0.169479,0.043322,...,0.011451,0.144048,0.102033,0.069941,0.009641,0.007079,0.129647,0.0,0.063585,0.122387,0.047020,0.055886,0.058342,0.033610,0.095249,0.047437,0.105421,0.037728,0.051045,0.184120,0.105999,0.049010,0.209392,0.062336,0.082067,0.093637,0.210657,0.019309,0.160776,0.186949,0.065328,0.116656,0.251105,0.056396,0.075940,0.187236,0.102819,0.109515,0.038805,0.086941
5,0.075410,0.020560,0.000000,0.121648,1.000000,0.164390,0.075932,0.302418,0.000000,0.000000,0.118213,0.053365,0.000000,0.135476,0.083971,0.063830,0.096110,0.095612,0.079224,0.032121,0.041732,0.019271,0.052776,0.076803,0.052181,0.091675,0.102966,0.090528,0.028561,0.000000,0.017516,0.232930,0.204039,0.038300,0.296278,0.030812,0.214088,0.195768,0.106027,0.173635,...,0.000000,0.057779,0.068374,0.189874,0.000000,0.000000,0.088809,0.0,0.159289,0.072059,0.075021,0.000000,0.061152,0.223592,0.000000,0.029744,0.127100,0.256836,0.096130,0.085791,0.000000,0.209454,0.146134,0.093192,0.085930,0.085811,0.068642,0.000000,0.054620,0.101157,0.090020,0.306634,0.103646,0.145354,0.119800,0.071333,0.064705,0.111755,0.159946,0.049236


In [30]:
movieId_grouped = train_df.groupby('movieId')
user_prediction_result_df = pd.DataFrame(index=list(movieId_grouped.indices.keys()), columns=user_sparse_matrix.index)
user_prediction_result_df

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
for movieId, group in tqdm(movieId_grouped):
    user_sim = user_cossim_df.loc[group['userId']]
    user_rating = group['rating']
    sim_sum = user_sim.sum(axis=0)

    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    user_prediction_result_df.loc[movieId] = pred_ratings

# return user_prediction_result_df.transpose()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [34]:
print(item_prediction_result_df.shape)
print(user_prediction_result_df.shape)
print(user_prediction_result_df.T.shape)

(610, 8938)
(8938, 610)
(610, 8938)


In [35]:
# 유저기반인지 아이템 기반인지는 같은 행렬에 기준이 어떤 것인지를 두고 하는 것

In [36]:
user_prediction_result_df.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
1,3.67043,3.34895,2.121,3.61963,3.76122,3.83087,3.77535,3.79756,3.17211,3.53956,3.77271,3.25295,3.38608,3.71767,3.67586,3.62024,3.68222,3.71063,3.7608,3.65439,3.7096,3.54082,3.46236,3.65635,3.54933,3.69778,3.70017,3.63817,3.57472,3.60583,3.69811,3.77944,3.66176,3.60127,3.67703,3.32949,3.73022,3.73684,3.66265,3.70991,...,3.23291,3.68141,3.73085,3.72981,3.26056,2.84388,3.66427,2.23791,3.73436,3.69662,3.5937,3.53617,3.56432,3.84746,3.47273,3.67719,3.77966,3.73643,3.71412,3.76155,3.56806,3.8059,3.72546,3.73731,3.41956,3.70841,3.68117,3.11601,3.70567,3.74232,3.69894,3.80129,3.66407,3.74929,3.73856,3.61476,3.74209,3.72009,3.70421,3.69463
2,3.01898,2.57281,1.27956,2.93753,3.21044,3.32198,3.10961,3.3381,2.35781,2.8281,3.19533,2.41644,2.58414,3.20225,3.05923,2.95389,3.08068,3.12762,3.13179,3.00293,3.13079,2.90958,2.70137,2.99499,2.76909,3.27871,3.13508,3.05045,2.94201,2.87687,3.01183,3.1045,3.04589,2.98701,3.14361,2.49345,3.25475,3.30884,2.98861,3.26457,...,2.33993,3.00907,3.18874,3.31376,2.41595,1.98107,3.00239,1.25203,3.03079,3.09173,2.91868,2.8896,2.92766,3.37873,2.69247,3.02139,3.03969,3.23124,3.1681,3.13769,2.79982,3.40795,3.13686,3.2123,2.61258,3.07227,3.03384,2.38314,3.1004,3.17795,2.96794,3.32972,2.95354,3.24571,3.15866,3.02316,3.11445,3.11634,3.28079,3.05471
3,3.02703,1.7197,1.26626,2.73966,2.92431,3.32667,2.83467,3.12298,1.68523,2.16925,3.03148,2.02516,2.12632,3.09258,2.77223,2.6048,2.84658,2.88318,3.04508,2.66741,2.69214,2.44399,2.4074,2.72977,1.9996,3.0305,2.85355,2.86627,2.57443,2.13558,2.86902,3.03456,2.85711,2.62894,2.82292,2.12788,3.02746,3.08393,2.79625,2.99991,...,2.03934,2.86468,2.91844,3.03235,1.87267,1.61107,2.88716,0.525583,2.78291,2.9425,2.20671,1.84724,2.52798,3.16183,2.37202,2.45175,2.87864,3.10433,2.91158,3.03315,2.50499,3.19994,2.9394,3.0869,2.22612,2.66949,2.96102,1.28233,2.91972,3.03434,2.42119,3.15846,2.86146,3.08181,2.78276,2.8544,2.98709,3.04798,3.03316,2.70117
4,0.636151,0.146062,0.0367179,0.72914,0.957915,1.21591,0.572114,1.16575,0.0990461,0.172367,1.01382,0.208322,0.191233,1.64497,0.439262,0.673211,0.758475,0.735548,0.754045,0.258811,0.344197,0.261046,0.379184,0.460082,0.161154,1.12963,0.574072,0.610436,0.491937,0.203056,0.686023,1.09784,0.93953,0.410265,0.943425,0.377929,1.06752,1.20179,0.616147,1.1298,...,0.108148,0.909392,0.552839,1.05536,0.21624,0.0974431,0.755014,0,0.403718,0.709331,0.304465,0.185581,0.419445,1.18062,0.245254,0.271887,0.854395,1.1071,1.15895,0.842544,0.239281,1.13412,0.808868,0.730212,0.463517,0.417435,0.866839,0.064485,0.703359,0.793243,0.419299,1.27232,0.834086,1.02853,0.438222,0.709787,0.836006,0.741379,1.00823,0.346251
5,2.33427,1.55944,0.466202,2.1024,2.54943,3.03136,2.35059,2.71027,1.19739,1.72931,2.67806,1.62483,1.57144,2.69429,2.21885,2.06364,2.31724,2.364,2.51215,1.95871,2.198,1.87701,1.82359,2.23057,1.66264,2.61757,2.29546,2.26701,2.00612,1.92448,2.61819,2.70976,2.35273,2.07908,2.52246,1.75008,2.62867,2.76508,2.21247,2.66091,...,1.33946,2.31166,2.41571,2.7132,1.27091,0.970138,2.33122,0.616505,2.27193,2.37832,1.73805,1.6173,1.96777,2.81019,1.64438,2.11667,2.36279,2.7084,2.58436,2.42826,1.8074,2.8605,2.44308,2.5553,1.67811,2.14235,2.38308,1.08121,2.40002,2.48269,1.94362,2.828,2.18487,2.71894,2.29913,2.15327,2.42895,2.48385,2.63636,2.13237


In [37]:
item_prediction_result_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,184257,184349,184471,184721,184791,184931,184987,184997,185033,185135,185435,185585,187031,187541,187593,187595,187717,188189,188301,188675,188797,189043,189111,189333,189547,190183,190207,190209,190213,190215,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
1,4.20279,4.18783,4.17609,3.03272,4.04561,4.24019,3.9933,3.90547,3.70279,4.18855,4.16638,3.96413,4.07492,3.89619,3.81881,4.2132,4.16727,4.13229,4.16556,4.1066,4.16574,4.086,4.14826,4.13404,4.13482,4.08018,3.71706,4.0126,4.19837,3.82467,4.09918,4.21487,4.21771,4.13656,4.03317,4.17642,0.863706,4.00858,4.03026,4.09802,...,2.81089,1.78243,3.99862,3.18195,4.07664,3.62958,3.62958,3.21241,3.62958,3.59316,3.74393,3.88679,3.12593,2.2081,3.96647,4.15315,3.18195,3.18195,3.24997,3.83627,3.62958,0.979003,0.979003,0,1.45119,2.40757,0.979003,0.979003,0.979003,0.979003,0.979003,0,0,0,0,0,0,0,0,1.38273
2,3.19261,3.15207,2.46138,0.280903,2.61916,2.9754,1.85877,1.68768,1.09268,3.02235,2.54129,2.51385,1.61082,0.758551,1.63399,3.18457,2.53038,2.90807,3.22072,2.92756,2.40354,2.14887,2.35921,2.71687,2.21494,2.23787,1.19091,1.83372,2.29285,0.196298,2.65615,3.05432,2.98765,2.38185,2.62798,2.91054,0,1.71507,1.77847,2.03524,...,2.61665,0,2.69157,1.20283,2.21043,1.95442,1.95442,1.91231,1.95442,2.76609,2.94477,2.44461,1.33566,1.90164,3.10808,2.94114,1.20283,1.20283,2.26655,2.28051,1.95442,0.387743,0.387743,1.70885,0.362692,1.91228,0.387743,0.387743,0.387743,0.387743,0.387743,1.70885,1.70885,1.70885,1.70885,1.70885,1.70885,1.70885,1.70885,2.32666
3,1.32315,1.20446,1.39635,0.217016,0.819716,1.59889,0.802514,0.755008,0.86708,1.55365,0.942893,1.34004,0.620453,0.450938,0.886356,1.34587,0.85248,1.39133,1.26625,0.999873,1.08345,1.13544,0.846797,1.04494,0.862301,0.817965,0.377594,0.406655,1.5742,1.38647,1.15163,1.46737,1.11338,0.947678,1.03943,1.18457,0,0.873352,0.950293,0.834913,...,0.0383185,0.224785,1.08515,0,0.807464,0.715827,0.715827,0.458293,0.715827,0.55052,0.314237,0.822655,0.469606,0.034753,0.784118,1.03524,0,0,0.397776,0.619607,0.715827,0,0,0,0.213525,0.115378,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3.48994,3.48805,3.38864,3.15017,3.30488,3.50267,3.31794,2.90629,3.03413,3.41697,3.42383,3.2559,3.18772,3.38452,3.20826,3.46465,3.51254,3.44664,3.41673,3.30509,3.5149,3.26365,3.33294,3.42041,3.46056,3.48881,3.06763,3.43736,3.58361,3.47541,3.26032,3.47623,3.49028,3.51451,3.43002,3.46574,0.492723,3.33497,3.31457,3.4271,...,2.43625,1.63485,3.11562,3.34077,3.24972,2.96916,2.96916,2.62972,2.96916,2.7532,3.08765,3.18312,2.56696,1.22959,3.22873,3.32863,3.34077,3.34077,2.62846,3.23077,2.96916,1.0335,1.0335,0.505535,0.810046,1.89312,1.0335,1.0335,1.0335,1.0335,1.0335,0.505535,0.505535,0.505535,0.505535,0.505535,0.505535,0.505535,0.505535,1.963
5,3.27916,3.1278,2.99947,2.56582,2.94179,3.16869,3.00793,2.67809,2.27668,3.01557,3.12409,2.23444,2.59869,3.02734,2.68661,3.08962,3.16745,2.82838,3.00623,2.50037,3.22707,2.88812,2.82624,2.87902,3.15316,2.91281,2.45999,2.90752,3.05304,2.43656,2.9588,3.18185,3.22697,3.31433,2.28735,3.17345,0.158787,2.73675,2.10767,2.91458,...,0.854196,0.33135,2.04168,1.80091,2.33965,1.83931,1.83931,1.53293,1.83931,1.65368,1.70121,2.0751,1.23932,0.318296,2.13799,2.66672,1.80091,1.80091,1.52717,1.83851,1.83931,0.391638,0.391638,0,0.460137,0.680643,0.391638,0.391638,0.391638,0.391638,0.391638,0,0,0,0,0,0,0,0,0.562705


In [38]:
user_prediction_result_df = user_prediction_result_df.T

In [39]:
user_prediction_result_df.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,184257,184349,184471,184721,184791,184931,184987,184997,185033,185135,185435,185585,187031,187541,187593,187595,187717,188189,188301,188675,188797,189043,189111,189333,189547,190183,190207,190209,190213,190215,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.67043,3.01898,3.02703,0.636151,2.33427,3.65344,2.54601,1.01293,1.59897,3.2986,3.27922,1.44417,1.68928,2.39352,1.655,3.61845,3.16598,2.33408,2.49103,1.87302,3.1424,2.48264,1.97355,2.47635,3.30969,2.15047,1.38065,2.06932,3.25523,0.878863,2.48965,3.79096,3.46025,3.10977,0.534037,2.97865,0.160365,2.27101,1.65699,1.658,...,0.189211,0.0695926,0.982785,0.37636,0.487489,0.649223,0.486917,0.379727,0.324611,0.945736,0.423192,0.745072,0.405764,0.513972,2.19388,1.12386,0.47045,0.423405,0.552659,0.393975,0.649223,0.163631,0.196357,0,0.035278,0.253466,0.0981783,0.261809,0.0654522,0.0981783,0.0654522,0,0,0,0,0,0,0,0,0.140421
2,3.34895,2.57281,1.7197,0.146062,1.55944,2.9715,1.49602,0.289957,0.408521,2.88504,2.30317,0.870564,0.640651,0.537099,0.796427,3.25767,2.21466,1.4527,2.29808,1.27025,2.14478,1.48337,1.08768,1.41437,2.05836,0.994619,0.57094,0.838371,1.72462,0.0264059,1.95155,3.36341,2.66866,2.1904,0.274587,2.48623,0,0.810158,0.341058,0.609998,...,0.427142,0,0.729262,0.12224,0.189323,0.317785,0.238339,0.350336,0.158893,1.12983,0.331821,0.393474,0.198616,0.870536,2.24052,0.880949,0.152799,0.137519,0.574945,0.256489,0.317785,0.115882,0.139058,0.379994,0.0198483,0.322547,0.0695291,0.185411,0.0463527,0.0695291,0.0463527,0.427493,0.332495,0.284995,0.379994,0.379994,0.332495,0.379994,0.332495,0.592074
3,2.121,1.27956,1.26626,0.0367179,0.466202,2.1174,0.780114,0.104695,0.218952,1.8424,0.993159,0.435337,0.189969,0.244504,0.337941,1.66166,0.936358,0.760682,0.959954,0.226082,1.31429,0.681447,0.28208,0.703348,0.947775,0.3694,0.194832,0.116562,1.24351,0.195957,0.607949,2.32485,1.65991,1.02633,0.0721052,1.34373,0,0.333002,0.295797,0.23367,...,0.00420843,0.00917187,0.159742,0,0.0689231,0.0962645,0.0721984,0.045983,0.0481323,0.0700665,0.0135367,0.102644,0.0601653,0.0117214,0.267391,0.157642,0,0,0.0588577,0.0367887,0.0962645,0,0,0,0.00627045,0.0154309,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3.61963,2.93753,2.73966,0.72914,2.1024,3.50549,2.50204,0.613417,1.2701,3.09378,3.09397,1.16455,1.52444,2.42512,1.39909,3.44215,3.3856,2.11603,2.37934,1.46228,3.1781,2.33222,1.61538,2.28041,3.33074,2.0975,1.16182,2.06787,3.17663,1.11919,2.31587,3.67758,3.41076,3.23664,0.413099,2.98251,0.0987468,2.19878,1.25164,1.58467,...,0.200076,0.0958159,0.691585,0.522647,0.478755,0.478978,0.359233,0.34248,0.239489,0.658025,0.249232,0.535239,0.424831,0.286039,1.79539,0.932784,0.653309,0.587978,0.504832,0.33195,0.478978,0.143633,0.172359,0.046584,0.0226976,0.232511,0.0861797,0.229813,0.0574532,0.0861797,0.0574532,0.052407,0.040761,0.034938,0.046584,0.046584,0.040761,0.046584,0.040761,0.251308
5,3.76122,3.21044,2.92431,0.957915,2.54943,3.45657,2.85021,1.20054,1.5878,3.13192,3.45784,0.990685,1.51991,2.89734,2.03389,3.44849,3.43953,1.97965,2.39822,1.38288,3.30462,2.71118,2.10526,2.21202,3.33802,2.39322,1.77773,1.86975,2.80939,0.442354,2.70269,3.7194,3.6376,3.44322,0.217809,3.03048,0.111525,2.27823,1.03021,1.74521,...,0.114544,0.0363891,0.465496,0.270785,0.233405,0.314517,0.235888,0.316118,0.157259,0.511364,0.18554,0.377933,0.196573,0.198374,1.50087,0.559401,0.338481,0.304633,0.397264,0.205775,0.314517,0.146712,0.176055,0,0.0228423,0.114604,0.0880273,0.234739,0.0586848,0.0880273,0.0586848,0,0,0,0,0,0,0,0,0.142512


## RMSE로 성능평가하기

In [40]:


def evaluate(test_df, prediction_result_df):
  groups_with_movie_ids = test_df.groupby(by='movieId')
  groups_with_user_ids = test_df.groupby(by='userId')
  intersection_movie_ids = sorted(list(set(list(prediction_result_df.columns)).intersection(set(list(groups_with_movie_ids.indices.keys())))))
  intersection_user_ids = sorted(list(set(list(prediction_result_df.index)).intersection(set(groups_with_user_ids.indices.keys()))))

  print(len(intersection_movie_ids))
  print(len(intersection_user_ids))

  compressed_prediction_df = prediction_result_df.loc[intersection_user_ids][intersection_movie_ids] # 해당되는 것만 사용하기 
  # compressed_prediction_df

  # test_df에 대해서 RMSE 계산
  grouped = test_df.groupby(by='userId')
  result_df = pd.DataFrame(columns=['rmse'])
  for userId, group in tqdm(grouped):
      if userId in intersection_user_ids:
          pred_ratings = compressed_prediction_df.loc[userId][compressed_prediction_df.loc[userId].index.intersection(list(group['movieId'].values))]
          pred_ratings = pred_ratings.to_frame(name='rating').reset_index().rename(columns={'index':'movieId','rating':'pred_rating'})
          actual_ratings = group[['rating', 'movieId']].rename(columns={'rating':'actual_rating'})

          final_df = pd.merge(actual_ratings, pred_ratings, how='inner', on=['movieId'])
          final_df = final_df.round(4) # 반올림

          # if not final_df.empty:
          #     rmse = sqrt(mean_squared_error(final_df['rating_actual'], final_df['rating_pred']))
          #     result_df.loc[userId] = rmse
          #     # print(userId, rmse)
    
  return final_df

In [41]:
evaluate(test_df, user_prediction_result_df)

4385
610


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


,actual_rating,movieId,pred_rating
0,5.0,3527,3.28567
1,3.5,84772,2.21159
2,3.5,103141,2.83544
3,4.0,81132,0.450808
4,4.5,130634,1.02244
...,...,...,...
218,4.0,106100,2.86823
219,4.0,111759,3.51395
220,1.0,4852,0.202402
221,3.0,2628,2.93777


In [43]:
evaluate(test_df, item_prediction_result_df)

4385
610


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


,actual_rating,movieId,pred_rating
0,5.0,3527,3.79741
1,3.5,84772,3.73065
2,3.5,103141,3.6897
3,4.0,81132,3.33402
4,4.5,130634,3.4512
...,...,...,...
218,4.0,106100,3.72969
219,4.0,111759,3.75842
220,1.0,4852,3.64888
221,3.0,2628,3.81238


In [46]:
# RMSE 값
result_df = evaluate(test_df, user_prediction_result_df)
print(result_df)
print(f"RMSE: {np.sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

# 유저베이스는 성능이 별로인 것 같음

4385
610


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



     actual_rating  movieId pred_rating
0              5.0     3527     3.28567
1              3.5    84772     2.21159
2              3.5   103141     2.83544
3              4.0    81132    0.450808
4              4.5   130634     1.02244
..             ...      ...         ...
218            4.0   106100     2.86823
219            4.0   111759     3.51395
220            1.0     4852    0.202402
221            3.0     2628     2.93777
222            5.0     1953     3.17222

[223 rows x 3 columns]
RMSE: 1.6949489761800296


In [47]:
# RMSE 값
result_df = evaluate(test_df, item_prediction_result_df)
print(result_df)
print(f"RMSE: {np.sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

# 아이템 베이스가 성능이 더 좋은 걸로 나타남

4385
610


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



     actual_rating  movieId pred_rating
0              5.0     3527     3.79741
1              3.5    84772     3.73065
2              3.5   103141      3.6897
3              4.0    81132     3.33402
4              4.5   130634      3.4512
..             ...      ...         ...
218            4.0   106100     3.72969
219            4.0   111759     3.75842
220            1.0     4852     3.64888
221            3.0     2628     3.81238
222            5.0     1953     3.86646

[223 rows x 3 columns]
RMSE: 0.8145193961484049


In [ ]:
# key point
# - sparse matrix를 만들어 기준이 되는 아이템들을 가지고 코사인 유사도를 구함
# - 그 코사인 유사도를 바탕으로 실제 데이터의 평점들과 코사인 매트릭스를 가지고 매트릭스 곱을 한 후 
# 아이템 기준 또는 유저 기준으로 예측을 함함